<a href="https://colab.research.google.com/github/SammyAlbataineh/Sports-Probability-Website/blob/main/NFL_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kagglehub transformers timm -q

import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import kagglehub

In [ ]:
import kagglehub

path = kagglehub.dataset_download("tobycrabtree/nfl-scores-and-betting-data")
print("Downloaded path:", path)

In [ ]:
import pandas as pd

# Construct the full path to the desired CSV file
scores_csv_path = os.path.join(path, 'spreadspoke_scores.csv')

# Load the CSV file into a pandas DataFrame
df_scores = pd.read_csv(scores_csv_path)

# Display the first 5 rows of the DataFrame
display(df_scores.head())

In [ ]:
df_scores = df_scores.drop(columns=['weather_detail'])
display(df_scores.head())

In [ ]:
print(path)
for root, dirs, files in os.walk(path):
    for name in files:
        print(os.path.join(root, name))

In [ ]:
df_scores['home_win'] = (df_scores['score_home'] > df_scores['score_away']).astype(int)
y = df_scores[['score_home', 'score_away', 'home_win']]
x = df_scores.drop(columns=['score_home', 'score_away', 'home_win'])
display(df_scores.head())

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
x_encoded = encoder.fit_transform(x[x.columns])

In [ ]:
print(df_scores.isnull().sum())

In [ ]:
median_score_home = df_scores['score_home'].median()
median_score_away = df_scores['score_away'].median()

df_scores['score_home'].fillna(median_score_home, inplace=True)
df_scores['score_away'].fillna(median_score_away, inplace=True)

print(f"Median score home: {median_score_home}")
print(f"Median score away: {median_score_away}")
print("Missing values after filling:")
print(df_scores[['score_home', 'score_away']].isnull().sum())

In [ ]:
median_score_home = df_scores['score_home'].median()
median_score_away = df_scores['score_away'].median()

df_scores['score_home'] = df_scores['score_home'].fillna(median_score_home)
df_scores['score_away'] = df_scores['score_away'].fillna(median_score_away)

print(f"Median score home: {median_score_home}")
print(f"Median score away: {median_score_away}")
print("Missing values after filling:")
print(df_scores[['score_home', 'score_away']].isnull().sum())

In [ ]:
print("Missing values in 'score_home' and 'score_away' after imputation:")
print(df_scores[['score_home', 'score_away']].isnull().sum())

print("\nMissing values in the entire df_scores DataFrame:")
print(df_scores.isnull().sum())

In [ ]:
display(df_scores.head())

In [ ]:
y_target = y['home_win']
X = x_encoded

print("Shape of y_target:", y_target.shape)
print("Shape of X:", X.shape)

In [ ]:
latest_season = df_scores['schedule_season'].max()
weights = (df_scores['schedule_season'] - df_scores['schedule_season'].min()) / \
          (latest_season - df_scores['schedule_season'].min()) + 0.1 # Add a small constant to ensure non-zero weights

print("First 5 calculated weights:")
print(weights.head())
print("Last 5 calculated weights:")
print(weights.tail())
print("Shape of weights:", weights.shape)

In [ ]:
cutoff_season = 2023

train_mask = df_scores['schedule_season'] <= cutoff_season
test_mask = df_scores['schedule_season'] > cutoff_season

X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y_target[train_mask], y_target[test_mask]
weights_train, weights_test = weights[train_mask], weights[test_mask]

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"Max season in training set: {df_scores[train_mask]['schedule_season'].max()}")
print(f"Min season in test set: {df_scores[test_mask]['schedule_season'].min()}")

In [ ]:
cutoff_season = 2023

train_mask = df_scores['schedule_season'] <= cutoff_season
test_mask = df_scores['schedule_season'] > cutoff_season

X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y_target[train_mask], y_target[test_mask]
weights_train, weights_test = weights[train_mask], weights[test_mask]

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"Max season in training set: {df_scores[train_mask]['schedule_season'].max()}")
print(f"Min season in test set: {df_scores[test_mask]['schedule_season'].min()}")

In [ ]:
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")
print(f"Shape of weights_train: {weights_train.shape}")
print(f"Shape of weights_test: {weights_test.shape}")

In [ ]:
model = LogisticRegression(solver='liblinear', random_state=42, max_iter=2000)
model.fit(X_train, y_train, sample_weight=weights_train)

print("Logistic Regression model successfully fitted to the training data with time-decaying weights.")

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy on Test Set: {accuracy:.4f}")